# import data

In [1]:
import pandas as pd
import numpy as np

MI = pd.read_csv('MI.csv',header=2)
MI.head()

,Obs #,Size,Volatility,POV,LnMktCap,Cost_bp
0,1,0.12,0.37,0.36,16.48,115.41
1,2,0.01,0.32,0.04,15.09,9.06
2,3,0.08,0.46,0.22,21.70,81.57
3,4,0.21,0.37,0.36,28.53,139.22
4,5,0.11,0.43,0.09,20.22,43.22


In [2]:
MI.shape

(2500, 6)

In [3]:
# give a random-state

In [4]:
r = np.random.RandomState(1234)
MI_s = MI.sample(n=10,replace =True,random_state=r)
MI_s

,Obs #,Size,Volatility,POV,LnMktCap,Cost_bp
723,724,0.18,0.34,0.29,27.80,62.40
1318,1319,0.12,0.38,0.43,16.99,76.90
664,665,0.23,0.37,0.25,26.74,110.49
2041,2042,0.24,0.41,0.20,18.05,98.99
279,280,0.14,0.48,0.09,22.12,47.65
1257,1258,0.07,0.39,0.23,18.73,64.51
30,31,0.09,0.35,0.10,24.02,27.83
1182,1183,0.10,0.34,0.25,21.62,72.65
1398,1399,0.12,0.45,0.15,17.67,67.15
2490,2491,0.34,0.42,0.47,22.10,130.61


# minimize function

In [5]:
from scipy.optimize import least_squares
import matplotlib.pyplot as plt
%matplotlib inline

def solve(sample):
    Y = sample['Cost_bp'].values
    size = sample['Size'].values
    volatility = sample['Volatility'].values
    POV = sample['POV'].values
    LnMktCap = sample['LnMktCap'].values
    xdata = np.array([size,volatility,POV, LnMktCap])
    #print xdata[0]
    #print xdata

    def model(x, coeffs):
        s = np.array(x[0])
        v = np.array(x[1])
        P = np.array(x[2])
        L = np.array(x[3])
   
        return coeffs[0]*(s**coeffs[1])*(v **coeffs[2])*(L**coeffs[4]) * (coeffs[5]*(P **coeffs[3])+ (1-coeffs[5]))

        #coeffs[0] is a1
        #coeffs[1] is a2
        #coeffs[2] is a3
        #coeffs[3] is a4
        #coeffs[4] is a5
        #coeffs[5] is b1

    x0 = np.array([500,0.5,0.5,0.5,0.5,0.5], dtype=float)  # An approximative initial solution that we can find from looking at the graph is for instance:

    def residuals(coeffs,x ,y):
        return model(x, coeffs)-y

    result = least_squares(residuals, x0, bounds=([0,0,0,0,0,0],[2000,1,1,1,1,1]), args=(xdata,Y))
    #print result.x, result.cost, result.optimality
    #print result.x
    #print Y
    #print  model(xdata, result.x )
    #plt.plot(Y,"o--", model(xdata, result.x ), '-')
    #print result
    return result.x



print solve(MI_s)
#solve(MI_s)

[  3.92403839e+02   4.21249141e-01   4.18507505e-01   3.15791093e-01
   3.76585832e-11   1.00000000e+00]


# with replacement

In [6]:
df = pd.DataFrame(columns=['a1','a2','a3','a4','a5','b1'])
for i in xrange(250):
    MI_s = MI.sample(n=250,replace =True)
    result = pd.DataFrame(np.reshape(solve(MI_s),(1,6)),columns=['a1','a2','a3','a4','a5','b1'])
    df = df.append(result)
df = df.reset_index(drop=True)
df.head()

,a1,a2,a3,a4,a5,b1
0,726.490125,0.328827,0.552606,0.981830,3.752258e-16,0.886290
1,705.071941,0.351131,0.458508,0.908927,3.331109e-12,0.934527
2,807.088316,0.368955,0.642362,0.821339,9.274177e-12,0.943762
3,719.185704,0.369925,0.534816,0.794140,1.932233e-02,0.991930
4,701.126783,0.337629,0.602684,0.698449,2.226212e-12,1.000000


# Table with Parameter Values, Standard Errors, T-Stat  for the full model.

In [7]:
summary1 = df.describe()
se =  summary1.ix['std'].values/summary1.ix['count'].values
se = pd.DataFrame(np.reshape(se,(1,6)),index=['se'],columns=['a1','a2','a3','a4','a5','b1'])
summary1 = summary1.append(se)
t = summary1.ix['mean'].values/summary1.ix['se'].values
t = pd.DataFrame(np.reshape(t,(1,6)),index=['t'],columns=['a1','a2','a3','a4','a5','b1'])
summary1 = summary1.append(t)
summary1

,a1,a2,a3,a4,a5,b1
count,250.000000,250.000000,250.000000,250.000000,2.500000e+02,250.000000
mean,647.278060,0.343034,0.565178,0.768070,3.318924e-02,0.968326
std,114.534698,0.020130,0.091893,0.102147,4.404277e-02,0.033181
min,282.107281,0.287760,0.276657,0.608463,2.132905e-26,0.883816
25%,568.996001,0.329276,0.514015,0.684865,2.028790e-12,0.943783
50%,652.096972,0.343180,0.566526,0.744886,1.084293e-02,0.974728
75%,721.067594,0.357246,0.621975,0.828712,5.968484e-02,1.000000
max,938.025179,0.396831,0.835585,1.000000,2.351305e-01,1.000000
se,0.458139,0.000081,0.000368,0.000409,1.761711e-04,0.000133
t,1412.842732,4260.161019,1537.605244,1879.824907,1.883921e+02,7295.705368


# without replacement

In [8]:
df2 = pd.DataFrame(columns=['a1','a2','a3','a4','a5','b1'])
for i in xrange(250):
    MI_s = MI.sample(n=250,replace =False)
    result = pd.DataFrame(np.reshape(solve(MI_s),(1,6)),columns=['a1','a2','a3','a4','a5','b1'])
    df2 = df2.append(result)
df2 = df2.reset_index(drop=True)
df2.head()

,a1,a2,a3,a4,a5,b1
0,721.589327,0.333793,0.537734,0.865443,9.175533e-12,0.932386
1,536.223256,0.346745,0.524048,0.672616,5.917400e-02,1.000000
2,570.313618,0.324312,0.435956,0.665297,2.183276e-14,1.000000
3,585.046572,0.349455,0.547214,0.701666,3.856812e-02,0.980688
4,551.936692,0.364098,0.541045,0.700204,7.846405e-02,1.000000


In [9]:
summary2 = df2.describe()
se =  summary2.ix['std'].values/summary2.ix['count'].values
se = pd.DataFrame(np.reshape(se,(1,6)),index=['se'],columns=['a1','a2','a3','a4','a5','b1'])
summary2 = summary2.append(se)
t = summary2.ix['mean'].values/summary2.ix['se'].values
t = pd.DataFrame(np.reshape(t,(1,6)),index=['t'],columns=['a1','a2','a3','a4','a5','b1'])
summary2 = summary2.append(t)
summary2

,a1,a2,a3,a4,a5,b1
count,250.000000,250.000000,250.000000,250.000000,2.500000e+02,250.000000
mean,649.015310,0.342495,0.558676,0.765639,2.860159e-02,0.968960
std,106.868607,0.017393,0.095577,0.100457,3.766892e-02,0.032980
min,358.336633,0.282891,0.287142,0.614721,4.991324e-30,0.886262
25%,577.661584,0.329734,0.493662,0.683012,1.928695e-13,0.942264
50%,647.065533,0.342210,0.561598,0.740977,7.656376e-03,0.979034
75%,721.796453,0.354276,0.610822,0.828655,4.954488e-02,1.000000
max,983.722514,0.392738,0.830129,1.000000,1.578553e-01,1.000000
se,0.427474,0.000070,0.000382,0.000402,1.506757e-04,0.000132
t,1518.255292,4922.785780,1461.327754,1905.391940,1.898222e+02,7345.070123


# a5(LnMktCap) is insignificant, new function without LnMktCap

In [10]:

from scipy.optimize import least_squares
import matplotlib.pyplot as plt
%matplotlib inline

def solve(sample):
    Y = sample['Cost_bp'].values
    size = sample['Size'].values
    volatility = sample['Volatility'].values
    POV = sample['POV'].values
    
    xdata = np.array([size,volatility,POV])
    #print xdata[0]
    #print xdata

    def model(x, coeffs):
        s = np.array(x[0])
        v = np.array(x[1])
        P = np.array(x[2])
        
   
        return coeffs[0]*(s**coeffs[1])*(v **coeffs[2]) * (coeffs[4]*(P **coeffs[3])+ (1-coeffs[4]))

        #coeffs[0] is a1
        #coeffs[1] is a2
        #coeffs[2] is a3
        #coeffs[3] is a4
        #coeffs[4] is b1


    x0 = np.array([500,0.5,0.5,0.5,0.5], dtype=float)  # An approximative initial solution that we can find from looking at the graph is for instance:

    def residuals(coeffs,x ,y):
        return model(x, coeffs)-y

    result = least_squares(residuals, x0, bounds=([0,0,0,0,0],[2000,1,1,1,1]), args=(xdata,Y))
    #print result.x, result.cost, result.optimality
    #print result.x
    #print Y
    #print  model(xdata, result.x )
    #plt.plot(Y,"o--", model(xdata, result.x ), '-')
    return result.x



print solve(MI_s)
#solve(MI_s)

[  8.27668858e+02   3.63623901e-01   7.04081282e-01   7.92726757e-01
   9.48990483e-01]


# with replacement without LnMktCap

In [11]:
df = pd.DataFrame(columns=['a1','a2','a3','a4','b1'])
for i in xrange(250):
    MI_s = MI.sample(n=250,replace =True)
    result = pd.DataFrame(np.reshape(solve(MI_s),(1,5)),columns=['a1','a2','a3','a4','b1'])
    df = df.append(result)
df = df.reset_index(drop=True)
df.head()

,a1,a2,a3,a4,b1
0,647.333357,0.338796,0.456010,0.801412,0.946112
1,556.018879,0.328588,0.389436,0.691674,0.989678
2,648.499015,0.310318,0.589810,0.673807,1.000000
3,669.680529,0.328569,0.503151,0.760801,0.981674
4,710.798168,0.367067,0.509193,0.836948,0.934723


In [12]:
summary = df.describe()
se =  summary.ix['std'].values/summary.ix['count'].values
se = pd.DataFrame(np.reshape(se,(1,5)),index=['se'],columns=['a1','a2','a3','a4','b1'])
summary = summary.append(se)
t = summary.ix['mean'].values/summary.ix['se'].values
t = pd.DataFrame(np.reshape(t,(1,5)),index=['t'],columns=['a1','a2','a3','a4','b1'])
summary = summary.append(t)
summary

,a1,a2,a3,a4,b1
count,250.000000,250.000000,250.000000,250.000000,250.000000
mean,700.922490,0.342953,0.564210,0.747702,0.975054
std,79.763234,0.018047,0.092819,0.093358,0.031943
min,550.158672,0.302247,0.345723,0.624357,0.890655
25%,638.417084,0.330227,0.488849,0.675322,0.950369
50%,689.935127,0.341793,0.561735,0.713760,0.996487
75%,762.795547,0.355423,0.633115,0.811482,1.000000
max,895.111366,0.400241,0.799614,1.000000,1.000000
se,0.319053,0.000072,0.000371,0.000373,0.000128
t,2196.884634,4750.770258,1519.654880,2002.246386,7631.169554


# without replacement without LnMktCap

In [13]:
df2 = pd.DataFrame(columns=['a1','a2','a3','a4','b1'])
for i in xrange(250):
    MI_s = MI.sample(n=250,replace =False)
    result = pd.DataFrame(np.reshape(solve(MI_s),(1,5)),columns=['a1','a2','a3','a4','b1'])
    df2 = df2.append(result)
df2 = df2.reset_index(drop=True)
df2.head()

,a1,a2,a3,a4,b1
0,731.222838,0.366576,0.576678,0.697216,1.000000
1,723.462503,0.358307,0.542557,0.707084,1.000000
2,736.098629,0.345028,0.612006,0.707068,1.000000
3,726.399031,0.354789,0.566848,0.711407,0.996327
4,673.788303,0.335806,0.573455,0.690237,1.000000


In [14]:
summary2 = df2.describe()
se =  summary2.ix['std'].values/summary2.ix['count'].values
se = pd.DataFrame(np.reshape(se,(1,5)),index=['se'],columns=['a1','a2','a3','a4','b1'])
summary2 = summary2.append(se)
t = summary2.ix['mean'].values/summary2.ix['se'].values
t = pd.DataFrame(np.reshape(t,(1,5)),index=['t'],columns=['a1','a2','a3','a4','b1'])
summary2 = summary2.append(t)
summary2

,a1,a2,a3,a4,b1
count,250.000000,250.000000,250.000000,250.000000,250.000000
mean,697.556720,0.343041,0.558069,0.746425,0.975826
std,76.036766,0.016497,0.090939,0.085835,0.030355
min,544.157038,0.301457,0.376614,0.613362,0.888820
25%,640.761994,0.332040,0.491608,0.683632,0.956004
50%,692.927423,0.343106,0.553685,0.716751,0.991505
75%,744.034223,0.354010,0.615178,0.781026,1.000000
max,928.650632,0.388752,0.781103,1.000000,1.000000
se,0.304147,0.000066,0.000364,0.000343,0.000121
t,2293.484984,5198.494362,1534.177609,2174.011236,8036.907186


# we take the average of two senerio (with and without replacement sampling)

# the market impact model will be 

In [15]:
coeffs = np.array([summary2.ix['mean'],summary.ix['mean']])
coeffs = list(np.mean(coeffs, axis=0))
print coeffs
def MI_model(x):
    "market impact function"
    
    s = np.array(x[0])          #size
    v = np.array(x[1])          #volatility
    P = np.array(x[2])          #POV
        
   
    return coeffs[0]*(s**coeffs[1])*(v **coeffs[2]) * (coeffs[4]*(P **coeffs[3])+ (1-coeffs[4]))


[699.23960524354425, 0.34299655134607687, 0.56113925539950404, 0.74706318505606117, 0.97544007537068878]
